# Predicting the Next Top Five Prize Money Owners' Name

## Team member : 
- Haoyu You 34735811
- Xiaomeng Zhou 
- Natalia He
- Ruodan Deng

## Introduction:

1. Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal.

In the field of professional sports, particularly in tennis, the ranking and earnings of players play a crucial role in assessing their success and market value. 

2. Clearly state the question you will try to answer with your project

How can players balance their achievement in the rank with their earnings in order to strive for higher prize money in their respective fields?


3. Identify and describe the dataset that will be used to answer the question

For our prediction task, we have chosen to utilize certain player attributes, namely age, current rank, best rank, and prize money, to forecast the next top five prize money owners in the sport of tennis. Age can provide insights into a player's experience and potential longevity in the sport, while current rank indicates their present performance level. Best rank reflects a player's career-high achievement in terms of ranking, highlighting their overall capabilities. The prize money refers to the monetary rewards earned by players through their performances in tournaments and it serves as a measure of a player's financial success and competitiveness in tournaments.


## Preliminary exploratory data analysis:

1. Demonstrate that the dataset can be read from the web into R 

Upload the dataset into data in R. Then use the relative path to read this dataset.


In [1]:
library(tidyverse)
library(dplyr)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

2. Clean and wrangle your data into a tidy format


In [2]:
player <- read_csv("player_stats.csv")
player

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


...1,Age,Country,Plays,Wikipedia,Current Rank,Best Rank,Name,Backhand,Prize Money,⋯,Facebook,Twitter,Nicknames,Grand Slams,Davis Cups,Web Site,Team Cups,Olympics,Weeks at No. 1,Tour Finals
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
0,26 (25-04-1993),Brazil,Right-handed,Wikipedia,378 (97),363 (04-11-2019),Oscar Jose Gutierrez,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,18 (22-12-2001),United Kingdom,Left-handed,Wikipedia,326 (119),316 (14-10-2019),Jack Draper,Two-handed,"$59,040",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,32 (03-11-1987),Slovakia,Right-handed,Wikipedia,178 (280),44 (14-01-2013),Lukas Lacko,Two-handed,"US$3,261,567",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
497,23 (14-03-1996),Netherlands,Left-handed,Wikipedia,495 (60),342 (05-08-2019),Gijs Brouwer,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
498,24 (17-05-1995),Ukraine,NA,Wikipedia,419 (81),419 (20-01-2020),Vladyslav Orlov,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
499,22 (26-03-1997),Tunisia,Left-handed,Wikipedia,451 (69),408 (24-12-2018),Aziz Dougaz,Two-handed,"$61,984",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [37]:
player_data <- player |>
               separate(col = `Age`, into = c("age","birth_date"), sep = "\\(") |>
               mutate(age = as.integer(age)) |>
               mutate(birth_date = str_remove(birth_date, "\\)")) |>
               separate(col = `Current Rank`, into = c("current_rank","current_score"), sep = "\\(") |>
               mutate(current_rank = as.integer(current_rank)) |>
               mutate(current_score = as.integer(str_remove(current_score, "\\)"))) |>
               mutate(prize_money = str_remove_all(`Prize Money`, "[A-Za-z$, ]")) |>
               select(Name, age, current_rank, prize_money) |>
               mutate(prize_money = as.integer(prize_money)) |>
               na.omit()
player_data

Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”


Name,age,current_rank,prize_money
<chr>,<int>,<int>,<int>
Jack Draper,18,326,59040
Lukas Lacko,32,178,3261567
Duck Hee Lee,21,236,374093
⋮,⋮,⋮,⋮
Artem Dubrivnyy,20,397,40724
Dan Added,20,382,57943
Aziz Dougaz,22,451,61984
